# In the name of god
# Persian Question Answering with a **BERT**
## by **Abolfazl Mohajeri**
This is a Persian Question Answering application of BERT.

First we import essential libraries.

As the transformers library has a large collection of pre-trained models, we use also use pre-trained model that has already been fine-tuned for PersianQA.

In [1]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import numpy as np
import pandas as pd
import torch

tokenizer = AutoTokenizer.from_pretrained("SajjadAyoubi/bert-base-fa-qa")
model = AutoModelForQuestionAnswering.from_pretrained("SajjadAyoubi/bert-base-fa-qa")

     |████████████████████████████████| 2.3MB 7.5MB/s 
     |████████████████████████████████| 901kB 36.2MB/s 
     |████████████████████████████████| 3.3MB 42.3MB/s 


Then we define function to get a question and abstract and return the answer of the given question based on the given abstract.

In [2]:
def fa_answer_question(question, abstract):
    input_ids = tokenizer.encode(question, abstract) # Encode question and abstract

    # Next 3 lines will seprate question and abstract
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a

    segment_ids = [0]*num_seg_a + [1]*num_seg_b # Create vector for two seprate segments

    start_scores, end_scores = model(torch.tensor([input_ids]),
                                     token_type_ids=torch.tensor([segment_ids]),
                                     return_dict=False) # Run our model

    # Find Answer
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]

    # Remove ## from answer
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    return answer

At the end we pass question and abstract to function.

In [3]:
# First Example
question = "پست خیابانی وقتی فوتبال بازی می کرد چی بود؟"
abstract = "جواد خیابانی پنجمین فرزند از میان خانواده‌ای با هشت فرزند است. پدر او انباردار اداره اصلاح بذر کرج بود. او تا ۶ سالگی مقیم کرج بود تا اینکه در سال ۱۳۵۱ به تهران آمد و شاگرد اول کل دبستان‌های منطقه ۱۰ آموزش و پرورش تهران شد. او در آزمون کنکور استعدادهای درخشان شرکت کرد و قبول شد اما به خاطر محدودیت‌های مالی نتوانست ثبت نام کند. او به علت بیماری فلج اطفال که در سال ۱۳۴۸ به آن دچار شده بود، مشکلات حرکتی زیادی داشت که هنوز هم از آن رنج می‌برد اما با این وجود در مسابقات دوچرخه‌سواری آموزشگاه‌های ایران در سال ۱۳۶۲ قهرمان شد. در سال‌های ۱۳۶۲ تا ۱۳۶۴ مدافع چپ و دروازه‌بان تیم فوتبال منتخب آموزشگاه‌های کرج بود. در سال ۱۳۶۶ از سوی تیم فوتبال منتخب دانشکده‌های فنی و مهندسی تهران در المپیک دانشجویان ایران در شیراز شرکت کرد و به مقام پنجم ایران در رشته فوتبال دست یافت. او به اردوی تیم ملی فوتبال دانشجویان ایران هم دعوت شد اما این اردو هرگز تشکیل نشد، زیرا در آن زمان ورزش دانشگاهی چندان فعالیت مهمی نداشت. جواد خیابانی در بیست و سوم اردیبهشت ۱۳۷۴ ازدواج کرد و یک فرزند دختر به نام «نگار» دارد"

fa_answer_question(question, abstract)

'مدافع چپ و دروازهبان'